In [ ]:
import re
import pandas as pd
import requests

import time #modulo para controlar los tiempos y los sleep (evitar el error 429)

#DEFINO LOS HEADERS: aqui tenemos las key de cada una para ir alternando
headers = {
#"X-RapidAPI-Key": "1ef9312677msh54e21139cb02deap1a8f37jsn53623375f3fe", #Elena
"X-RapidAPI-Key": "28a3c3ab32msha5ca6e00170b6c7p197d01jsn44c7f725210a", #Marga
#"X-RapidAPI-Key": "2583d39c0emshe321f51d17d9328p10cfb0jsn38ed4dc93da2", #Zahara
#"X-RapidAPI-Key": "47608e078dmsha9428889aafcc0ap1afc77jsn6e04b24d521c", #MariCarmen
"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

#Asignación: 
# MariCarmen = Drama + Comedy
# Zahara = Action + Fantasy 
# Elena = Horror + Mystery 
# Marga = thriller + Romance

#DEFINO LOS GENEROS 
#genres = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]
genres = ["Thriller"] #aqui hay que modificar el género por el cual estamos descargando 
#year == 1990 a 2023 
#genre == genres 
#page == de 1 a 50 
#titletype == movie or short 
#limit = 50 
sleep_time = 2

#SACAR TODAS LAS URLS: 

lista_peliculas = [] #Lista de peliculas fuera del bucle! 
for genre in genres:
  
    for year in range(1990, 2000): #aqui modificamos el rango de años. 
        #Cesar says: ponemos rango de 1 a 50 pq es el max permitido por la API en una sola call 
        for page in range (1, 51):  

            url = f"https://moviesdatabase.p.rapidapi.com/titles?year={year}&genre={genre}&page={page}&limit=50" #ponemos el límite de 50 que nos dijo Ana. 
            
            response = requests.get(url, headers=headers)
            #SI LA RESPUESTA ES 200 ENTONCES COMIENZO A HACER TODO EL LOOP DE LINKS Y EXTRAER DATOS: 
            if response.status_code == 200:
                # Procesamos la respuesta da data aqui, lo convertimos a JSONy extraemos la información:
                datos = response.json()
                lista_de_diccionarios = datos["results"]
                

                for datos in lista_de_diccionarios: 
                    
                    #lista que usamos para ir appendeando toda la información de abajo /cada peli y será una lista "de paso"
                    lista_consumible = []
                    
                    # aqui creamos las variables con las "rutas " que hemos ido buscando arriba /dentro del json 
                    tipo = datos["titleType"]["text"]
                    texto = datos["titleText"]["text"]
                    #para que no me de error el NONE de aquellas pelis que no tengan mes o año. añado el check de: 
                    release_date = datos.get("releaseDate")
                    if release_date:
                        mes = release_date.get("month")
                        anio = release_date.get("year")
                        #esto es para que no me añada las que tienen NONE, porque dijo Ana que las obviemos
                    else:
                       continue
                    #hasta aqui el control del mes y año 
                    id = datos["id"]
                    if tipo == "Movie" or tipo =="Short":

                        if mes:
                            lista_consumible.append(mes)
                            lista_consumible.append(genre) 
                            lista_consumible.append(tipo)
                            lista_consumible.append(texto)
                            lista_consumible.append(anio)
                            lista_consumible.append(id)
                        else:
                            continue 
                        
                    if lista_consumible: #evita hacer append de tuplas vacías (poq no tienen año o mes)
                        
                        lista_peliculas.append(tuple(lista_consumible))
                        #lista_peliculas.append(lista_consumible)
                    sleep_time
                print(f"Received data for {genre} in {year}, page {page}")

               
            #SI LA RESPUESTA NO ES 200. TELL ME SOMETHING I DONT KNOW
            elif response.status_code == 429:
                print("Too many requests. Sleeping...")


            else: # Otro error que no sea el 429 
                if "error" in response.text:
                    error_message = response.json().get("error")
                    print(f"Error Message: {error_message}")

            print("...")


   

In [ ]:
len(lista_peliculas) #Es el listado de tuplas que sacamos

In [ ]:
df_peliculas = pd.DataFrame(lista_peliculas) #convertirlo a data Frame

In [ ]:
df_peliculas = pd.DataFrame(lista_peliculas, columns=["genre", "type", "name", "month", "year", "id"]) #convertirlo a dataframe con las columnas que hemos descargado. Veréis q he añadido el género para que cuando lo juntesmo todo, sepamos q es


In [ ]:
df_peliculas

ESTOS SON LOS PASOS QUE TENDREMOS QUE HACER UNA VEZ DESCARGADO TODO. TAL COMO DECÏA ANA EN LA SESION, PASAREMOS TODOS LOS FIHCEROS A LA SCRUM MASTER PARA UQE LOS FUSIONE. 

In [ ]:
df_peliculas.to_csv("thriller_2013_5.csv") #poner el nombre de lo que estamos guardando (género + intervalo de años)

In [ ]:
df_peliculas.to_csv("datos_comedy.csv") #guardo cada descarga q haga con el nombre de lo q he descargado y siempre poniendo el .csv detrás. 
df_peliculas.to_csv("datos_romance.csv") #otras opcionnes = a la anterior 

In [ ]:
df_peliculas.to_excel("datos_api.xlsx") #si queremos guardarlo en excel en lugar de csv 

In [ ]:
#como lo juntamos todo? los CSVs que nos hemos descargado cada una: 

df_comedy = pd.read_csv("datos_comedy.csv")
df_drama = pd.read_csv("datos_drama.csv")

In [ ]:
df_final = pd.concat([df_comedy, df_drama], axis = 0)
#axis 0 == contactenar por columnas
#axis 1 == concatenar por filas 

In [ ]:
df_final.to_csv("datos_api_final.csv") #para convertirlo todo a csv 